# City scrapper

This notebook scraps Wikipedia route based on structured marked-up language patterns on the seek of city coordinates. There are plenty of applications, but the most promising is logistics.   

In [6]:
import requests as req
from bs4 import BeautifulSoup

host = 'https://en.wikipedia.org'
head_route = '/wiki'


In [7]:
tail_route = '/List_of_municipalities_of_Brazil'

route = head_route + tail_route
css_query = "#mw-content-text > div.mw-parser-output > table > tbody > tr > td > a"

r = req.get(host+route)
soup = BeautifulSoup(r.text, 'html.parser')

city_routes = []
for el in soup.select(css_query):
  city_routes.append(el.attrs['href'])

print(city_routes)


['/wiki/Assis_Brasil', '/wiki/Brasil%C3%A9ia', '/wiki/Epitaciol%C3%A2ndia', '/wiki/Xapuri', '/wiki/Acrel%C3%A2ndia', '/wiki/Bujari', '/wiki/Capixaba,_Acre', '/wiki/Pl%C3%A1cido_de_Castro,_Acre', '/wiki/Porto_Acre', '/wiki/Rio_Branco,_Acre', '/wiki/Senador_Guiomard', '/wiki/Manoel_Urbano', '/wiki/Santa_Rosa_do_Purus', '/wiki/Sena_Madureira', '/wiki/Cruzeiro_do_Sul,_Acre', '/wiki/M%C3%A2ncio_Lima', '/wiki/Marechal_Thaumaturgo', '/wiki/Porto_Walter', '/wiki/Rodrigues_Alves,_Acre', '/wiki/Feij%C3%B3,_Acre', '/wiki/Jord%C3%A3o,_Acre', '/wiki/Tarauac%C3%A1', '/wiki/Amap%C3%A1_(city)', '/wiki/Pracu%C3%BAba', '/wiki/Tartarugalzinho', '/wiki/Cal%C3%A7oene', '/wiki/Oiapoque', '/wiki/Cutias', '/wiki/Ferreira_Gomes', '/wiki/Itaubal', '/wiki/Macap%C3%A1', '/wiki/Pedra_Branca_do_Amapari', '/wiki/Porto_Grande', '/wiki/Santana,_Amap%C3%A1', '/wiki/Serra_do_Navio', '/wiki/Laranjal_do_Jari', '/wiki/Mazag%C3%A3o', '/wiki/Vit%C3%B3ria_do_Jari', '/wiki/Anam%C3%A3', '/wiki/Anori', '/wiki/Beruri', '/wiki/Caa

In [8]:
import json

tail_route = '/Assis_Brasil'
route = head_route + tail_route

url = host + route

r = req.get(url)
soup = BeautifulSoup(r.text)

scripts = str(soup.find('script').text).strip()


In [9]:
import json

code_equalities = [
    code_line.split('=') for code_line in scripts.split(';')
]

code_equalities = list(
    filter(lambda el: len(el) == 2, code_equalities)
)

lh_terms = list(
    set([ code_equality[0] for code_equality in code_equalities ])
)

lr_dict = {
    lh_term: [] for lh_term in lh_terms
}

for code_equality in code_equalities:
    #print(code_equality)
    lh_key = code_equality[0]
    
    lr_dict[lh_key].append(code_equality[1])


field_str = "wgCoordinates"
    
confs = list(
    filter(
        lambda rh_term: field_str in rh_term, 
        lr_dict['RLCONF']
    )
)

assert len(confs) == 1

conf = json.loads(confs[0])

lat_lng = (
    conf[field_str]['lat'],
    conf[field_str]['lon']
)

print(lat_lng)


(-10.940833333333334, -69.56694444444445)


In [16]:
host = 'https://en.wikipedia.org'

def brazilian_city_routes():    
    head_route = '/wiki'
    tail_route = '/List_of_municipalities_of_Brazil'
    route = head_route + tail_route
    
    uri = host+route
    
    css_query = "#mw-content-text > div.mw-parser-output > table > tbody > tr > td > a"

    r = req.get(uri)
    
    soup = BeautifulSoup(r.text, 'html.parser')

    for el in soup.select(css_query):
      yield el.attrs['href']

def get_latlng_tuple(url):
    r = req.get(url)
    soup = BeautifulSoup(r.text)

    scripts = str(soup.find('script').text).strip()

    from json import loads

    code_equalities = [
        code_line.split('=') for code_line in scripts.split(';')
    ]

    code_equalities = list(
        filter(lambda el: len(el) == 2, code_equalities)
    )

    lh_terms = list(
        set([ code_equality[0] for code_equality in code_equalities ])
    )

    lr_dict = {
        lh_term: [] for lh_term in lh_terms
    }

    for code_equality in code_equalities:
        #print(code_equality)
        lh_key = code_equality[0]
        lh_value = code_equality[1]
        
        lr_dict[lh_key].append(lh_value)


    field_str = "wgCoordinates"

    try:
        confs = list(
            filter(
                lambda rh_term: field_str in rh_term, 
                lr_dict['RLCONF']
            )
        )
        
    except:
       return () 

    if(len(confs) == 1):
        try: 
            conf = json.loads(confs[0])
            json.loads(confs[0])
        except:
            return ()
        
        return (
            conf[field_str]['lat'],
            conf[field_str]['lon']
        )
    else:
        return ()

In [17]:
import json

for route in brazilian_city_routes():
    uri = host + route
    
    coords = str(get_latlng_tuple(uri))
    
    print(uri + " " + coords)
        

https://en.wikipedia.org/wiki/Assis_Brasil (-10.940833333333334, -69.56694444444445)
https://en.wikipedia.org/wiki/Brasil%C3%A9ia (-11, -68.73333333333333)
https://en.wikipedia.org/wiki/Epitaciol%C3%A2ndia (-11.02888888888889, -68.74083333333333)
https://en.wikipedia.org/wiki/Xapuri (-10.651944444444444, -68.5038888888889)
https://en.wikipedia.org/wiki/Acrel%C3%A2ndia (-9.827777777777778, -66.88277777777778)
https://en.wikipedia.org/wiki/Bujari ()
https://en.wikipedia.org/wiki/Capixaba,_Acre (-10.572777777777777, -67.67583333333334)
https://en.wikipedia.org/wiki/Pl%C3%A1cido_de_Castro,_Acre (-10.275833333333335, -67.15)
https://en.wikipedia.org/wiki/Porto_Acre (-9.587777777777779, -67.53277777777778)
https://en.wikipedia.org/wiki/Rio_Branco,_Acre (-9.974722222222223, -67.81)
https://en.wikipedia.org/wiki/Senador_Guiomard (-10.150833333333333, -67.73583333333333)
https://en.wikipedia.org/wiki/Manoel_Urbano ()
https://en.wikipedia.org/wiki/Santa_Rosa_do_Purus (-9.432777777777778, -70.492

https://en.wikipedia.org/wiki/Porto_de_Moz,_Par%C3%A1 (-1.75, -52.233333333333334)
https://en.wikipedia.org/wiki/Faro,_Par%C3%A1 ()
https://en.wikipedia.org/wiki/Juruti,_Par%C3%A1 (-2.152222222222222, -56.092222222222226)
https://en.wikipedia.org/wiki/%C3%93bidos,_Par%C3%A1 (-1.9, -55.516666666666666)
https://en.wikipedia.org/wiki/Oriximin%C3%A1 (-1.7658333333333334, -55.865833333333335)
https://en.wikipedia.org/wiki/Terra_Santa (-2.103889, -56.486944)
https://en.wikipedia.org/wiki/Alenquer,_Par%C3%A1 (-1.944818, -54.732019)
https://en.wikipedia.org/wiki/Belterra (-2.6358333333333333, -54.93694444444444)
https://en.wikipedia.org/wiki/Curu%C3%A1 (-1.8877777777777778, -55.11694444444444)
https://en.wikipedia.org/wiki/Moju%C3%AD_dos_Campos (-2.6847222222222227, -54.640277777777776)
https://en.wikipedia.org/wiki/Monte_Alegre,_Par%C3%A1 ()
https://en.wikipedia.org/wiki/Placas (-3.8674, -54.2172)
https://en.wikipedia.org/wiki/Prainha,_Par%C3%A1 (-1.8, -53.480000000000004)
https://en.wikipedi

https://en.wikipedia.org/wiki/Paragominas (-2.9958333333333336, -47.35277777777778)
https://en.wikipedia.org/wiki/Rondon_do_Par%C3%A1 (-4.77583, -48.0669)
https://en.wikipedia.org/wiki/Ulian%C3%B3polis (-3.75583, -47.5008)
https://en.wikipedia.org/wiki/%C3%81gua_Azul_do_Norte (-6.7908333333333335, -50.466944444444444)
https://en.wikipedia.org/wiki/Cana%C3%A3_dos_Caraj%C3%A1s (-6.496944444444445, -49.87777777777778)
https://en.wikipedia.org/wiki/Curion%C3%B3polis (-6.090833333333333, -49.54083333333333)
https://en.wikipedia.org/wiki/Eldorado_dos_Caraj%C3%A1s (-6.10389, -49.355)
https://en.wikipedia.org/wiki/Parauapebas (-6.067777777777778, -49.901944444444446)
https://en.wikipedia.org/wiki/Pau_D%27Arco,_Par%C3%A1 (-7.832777777777777, -50.04388888888889)
https://en.wikipedia.org/wiki/Pi%C3%A7arra (-6.43778, -48.8719)
https://en.wikipedia.org/wiki/Reden%C3%A7%C3%A3o,_Par%C3%A1 (-8.02888888888889, -50.030833333333334)
https://en.wikipedia.org/wiki/Rio_Maria (-3.83778, -50.6378)
https://en.

https://en.wikipedia.org/wiki/Iracema,_Roraima (2.1819444444444445, -61.04083333333333)
https://en.wikipedia.org/wiki/Mucaja%C3%AD (2.4444444444444446, -60.919444444444444)
https://en.wikipedia.org/wiki/Caroebe (0.8838888888888888, -59.69583333333333)
https://en.wikipedia.org/wiki/Rorain%C3%B3polis (0.9458333333333333, -60.41777777777777)
https://en.wikipedia.org/wiki/S%C3%A3o_Jo%C3%A3o_da_Baliza (0.9508333333333333, -59.91083333333333)
https://en.wikipedia.org/wiki/S%C3%A3o_Luiz,_Roraima (1.0111, -60.0452)
https://en.wikipedia.org/wiki/Aragominas (-7.161944444444445, -48.52777777777778)
https://en.wikipedia.org/wiki/Aragua%C3%ADna (-7.190833333333334, -48.206944444444446)
https://en.wikipedia.org/wiki/Araguan%C3%A3,_Tocantins (-6.5808333333333335, -48.64388888888889)
https://en.wikipedia.org/wiki/Arapoema (-7.6577777777777785, -49.06388888888888)
https://en.wikipedia.org/wiki/Baba%C3%A7ul%C3%A2ndia (-7.205, -47.75694444444444)
https://en.wikipedia.org/wiki/Bandeirantes_do_Tocantins (-

https://en.wikipedia.org/wiki/Almas,_Tocantins (-11.573888888888888, -47.169999999999995)
https://en.wikipedia.org/wiki/Arraias (-12.930833333333332, -46.937777777777775)
https://en.wikipedia.org/wiki/Aurora_do_Tocantins (-12.712777777777777, -46.407777777777774)
https://en.wikipedia.org/wiki/Chapada_da_Natividade (-11.616944444444444, -47.75083333333333)
https://en.wikipedia.org/wiki/Combinado (-12.791944444444445, -46.538888888888884)
https://en.wikipedia.org/wiki/Concei%C3%A7%C3%A3o_do_Tocantins (-12.21888888888889, -47.297777777777775)
https://en.wikipedia.org/wiki/Dian%C3%B3polis ()
https://en.wikipedia.org/wiki/Lavandeira,_Tocantins (-12.78888888888889, -46.505833333333335)
https://en.wikipedia.org/wiki/Natividade,_Tocantins (-11.709722222222222, -47.723333333333336)
https://en.wikipedia.org/wiki/Novo_Alegre (-12.9308, -46.5739)
https://en.wikipedia.org/wiki/Novo_Jardim (-11.866666666666667, -46.56666666666667)
https://en.wikipedia.org/wiki/Paran%C3%A3,_Tocantins (-12.615, -47.88

...

https://en.wikipedia.org/wiki/Viadutos (-27.566666666666666, -52.016666666666666)
https://en.wikipedia.org/wiki/Alpestre,_Rio_Grande_do_Sul (-27.24888888888889, -53.035)
https://en.wikipedia.org/wiki/Ametista_do_Sul (-27.360833333333336, -53.18194444444444)
https://en.wikipedia.org/wiki/Cai%C3%A7ara,_Rio_Grande_do_Sul (-27.273888888888887, -53.43194444444444)
https://en.wikipedia.org/wiki/Constantina,_Rio_Grande_do_Sul (-27.733333333333334, -52.983333333333334)
https://en.wikipedia.org/wiki/Cristal_do_Sul (-27.453888888888887, -53.24583333333334)
https://en.wikipedia.org/wiki/Dois_Irm%C3%A3os_das_Miss%C3%B5es (-27.65888888888889, -53.530833333333334)
https://en.wikipedia.org/wiki/Engenho_Velho (-27.7077777878, -52.9127777878)
https://en.wikipedia.org/wiki/Erval_Seco (-27.55, -53.5)
https://en.wikipedia.org/wiki/Frederico_Westphalen (-27.366666666666667, -53.4)
https://en.wikipedia.org/wiki/Gramado_dos_Loureiros (-27.44388888888889, -52.91777777777777)
https://en.wikipedia.org/wiki/Ira%

https://en.wikipedia.org/wiki/Bossoroca (-28.729999999999997, -54.9)
https://en.wikipedia.org/wiki/Catu%C3%ADpe (-28.25, -54.011944444444445)
https://en.wikipedia.org/wiki/Dezesseis_de_Novembro (-28.224999999999998, -55.04888888888889)
https://en.wikipedia.org/wiki/Entre-Iju%C3%ADs (-28.360000000000003, -54.26777777777777)
https://en.wikipedia.org/wiki/Eug%C3%AAnio_de_Castro (-28.358333333333334, -54.14888888888889)
https://en.wikipedia.org/wiki/Giru%C3%A1 (-28.02777777777778, -54.35)
https://en.wikipedia.org/wiki/Pirap%C3%B3 (-28.045, -55.19888888888889)
https://en.wikipedia.org/wiki/Rolador (-28.25777777777778, -54.81777777777778)
https://en.wikipedia.org/wiki/Santo_%C3%82ngelo ()
https://en.wikipedia.org/wiki/Santo_Ant%C3%B4nio_das_Miss%C3%B5es (-28.510833333333334, -55.22777777777778)
https://en.wikipedia.org/wiki/S%C3%A3o_Luiz_Gonzaga (-28.407777777777778, -54.96083333333333)
https://en.wikipedia.org/wiki/S%C3%A3o_Miguel_das_Miss%C3%B5es (-28.55, -54.55277777777778)
https://en.wik

https://en.wikipedia.org/wiki/Angelina,_Santa_Catarina (-27.583333333333332, -48.983333333333334)
https://en.wikipedia.org/wiki/Canelinha (-27.283333333333335, -48.78333333333333)
https://en.wikipedia.org/wiki/Leoberto_Leal (-27.506944444444443, -49.286944444444444)
https://en.wikipedia.org/wiki/Major_Gercino (-27.4181, -48.9514)
https://en.wikipedia.org/wiki/Nova_Trento (-27.28666666666667, -48.930277777777775)
https://en.wikipedia.org/wiki/S%C3%A3o_Jo%C3%A3o_Batista,_Santa_Catarina (-27.27583333333333, -48.848888888888894)
https://en.wikipedia.org/wiki/Tijucas (-27.24138888888889, -48.63361111111111)
https://en.wikipedia.org/wiki/Bela_Vista_do_Toldo (-26.266666666666666, -50.5)
https://en.wikipedia.org/wiki/Canoinhas (-26.176944444444445, -50.39)
https://en.wikipedia.org/wiki/Irine%C3%B3polis (-26.2386, -50.7997)
https://en.wikipedia.org/wiki/Itai%C3%B3polis ()
https://en.wikipedia.org/wiki/Mafra,_Santa_Catarina (-26.116666666666667, -49.8)
https://en.wikipedia.org/wiki/Major_Vieira 

https://en.wikipedia.org/wiki/Tangar%C3%A1,_Santa_Catarina ()
https://en.wikipedia.org/wiki/Treze_T%C3%ADlias (-27.001944444444444, -51.405833333333334)
https://en.wikipedia.org/wiki/Vargem_Bonita (-20.17388888888889, -45.71083333333333)
https://en.wikipedia.org/wiki/Videira (-27.00777777777778, -51.151944444444446)
https://en.wikipedia.org/wiki/Anchieta,_Santa_Catarina (-26.53388888888889, -53.33083333333334)
https://en.wikipedia.org/wiki/Bandeirante,_Santa_Catarina ()
https://en.wikipedia.org/wiki/Barra_Bonita,_Santa_Catarina ()
https://en.wikipedia.org/wiki/Belmonte,_Santa_Catarina (-26.841388888888886, -53.57555555555556)
https://en.wikipedia.org/wiki/Descanso,_Santa_Catarina (-26.825833333333332, -53.50194444444445)
https://en.wikipedia.org/wiki/Dion%C3%ADsio_Cerqueira (-26.25, -53.63333333333333)
https://en.wikipedia.org/wiki/Guaraciaba,_Santa_Catarina (-26.598888888888887, -53.51777777777777)
https://en.wikipedia.org/wiki/Guaruj%C3%A1_do_Sul (-26.3853, -53.5278)
https://en.wikip

https://en.wikipedia.org/wiki/Imbituba (-28.233333333333334, -48.666666666666664)
https://en.wikipedia.org/wiki/Jaguaruna (-28.615, -49.0256)
https://en.wikipedia.org/wiki/Laguna,_Santa_Catarina (-28.482499999999998, -48.780833333333334)
https://en.wikipedia.org/wiki/Orleans,_Santa_Catarina (-28.35, -49.3)
https://en.wikipedia.org/wiki/Pedras_Grandes (-28.45, -49.18333333333333)
https://en.wikipedia.org/wiki/Pescaria_Brava (-28.383333333333333, -48.88333333333333)
https://en.wikipedia.org/wiki/Rio_Fortuna (-28.1, -49.11666666666667)
https://en.wikipedia.org/wiki/Sang%C3%A3o (-28.6378, -49.1292)
https://en.wikipedia.org/wiki/Santa_Rosa_de_Lima,_Santa_Catarina ()
https://en.wikipedia.org/wiki/S%C3%A3o_Ludgero (-28.3258, -49.1767)
https://en.wikipedia.org/wiki/S%C3%A3o_Martinho,_Santa_Catarina (-28.165, -48.97888888888889)
https://en.wikipedia.org/wiki/Treze_de_Maio (-28.5589, -49.1478)
https://en.wikipedia.org/wiki/Tubar%C3%A3o (-28.466666666666665, -49.00694444444444)
https://en.wikiped